In [1]:
from google.colab import drive
drive.mount('/content/drive')

root="/content/drive/MyDrive/signate/ワールドＡＩ/"
anomaly_root=root+"anomaly/"

import cv2
import os
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob
import json
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

train_path=root+"train/"
h=1080
w=1920

category={"要補修-1.区画線":0,
 "要補修-2.道路標識":1,
 "要補修-3.照明":2,
 "補修不要-1.区画線":0,
 "補修不要-2.道路標識":1,
 "補修不要-3.照明":2}

anomaly_data_path=anomaly_root+"anomaly_data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# データセット

In [ ]:
video_files = glob.glob(train_path+"*.mp4")
base_names=[]
for file_path in video_files:
    base_name=os.path.basename(file_path)[:-4]
    base_names.append(base_name)

In [ ]:
category_names=["要補修-1.区画線",
 "要補修-2.道路標識",
 "要補修-3.照明",
 "補修不要-1.区画線",
 "補修不要-2.道路標識",
 "補修不要-3.照明"]
for n in category_names:
    os.makedirs(anomaly_data_path+n, exist_ok=True)

In [ ]:
for bn in base_names:
    with open(f'{train_path}{bn}.json') as f:
        data_list = json.load(f)
    cap = cv2.VideoCapture(f"{train_path}{bn}.mp4")
    for data in data_list:
        frame_id=data["frame_id"]
        labels=data["labels"]
        ret, frame = cap.read()
        im=Image.fromarray(frame)
        for k,boxes in labels.items():
            for i,box in enumerate(boxes):
                im.crop((box[0][0],box[0][1],box[1][0],box[1][1])).save(f'{anomaly_data_path}{k}/crop_{bn}_{frame_id}_{i}.jpg', quality=95)

In [ ]:
glob.glob(train_path+"*.mp4")

In [ ]:
img_paths=glob.glob(anomaly_data_path+"*/*/*.jpg")
for p in img_paths:
    img = Image.open(p)
    img_resize = img.resize((224, 224))
    img_resize.save(p)

In [ ]:
for i,n in enumerate(["区画線","道路標識","照明"]):
    for ano in ["要補修-","補修不要-"]:
        file_num=len(glob.glob((f"{anomaly_data_path}{n}/{ano}{i+1}.{n}/*")))
        print(f"{ano}{i+1}.{n}:{file_num}")


要補修-1.区画線:17771
補修不要-1.区画線:7101
要補修-2.道路標識:223
補修不要-2.道路標識:4521
要補修-3.照明:242
補修不要-3.照明:5871


# 学習

In [2]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchinfo import summary
import matplotlib.pyplot as plt
import time
import os 
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#cuda:0

cuda:0


In [6]:
def tl(category):
#category="道路標識"
    data_transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    data_dir = anomaly_data_path+category
    image_dataset = datasets.ImageFolder(data_dir,data_transform)
                    
    train_dataset, val_dataset = torch.utils.data.random_split(
        image_dataset, lengths=[0.9,0.1], generator=torch.Generator().manual_seed(42)
    )
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=600,
                                                shuffle=True, num_workers=2)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=600,
                                                shuffle=True, num_workers=2)
    dataloaders={"train":train_dataloader,"val":val_dataloader}

    dataset_sizes = {"train": len(train_dataset), 'val':len(val_dataset)}
    class_names = image_dataset.classes
    class_num=len(class_names)
    print(class_names)

    model = models.efficientnet_v2_s(pretrained=True)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=class_num, bias=True)
    )

    for param in model.parameters():
        param.requires_grad = False

    last_layers = list(model.children())[-2:]
    for last_layer in last_layers:
        for param in last_layer.parameters():
            param.requires_grad = True

    #print(model)
    model=torch.load(os.path.join(anomaly_root,f"{category}_7.pth"))
    print(summary(model=model))



    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    since = time.time()


    num_epochs=100
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #tensor(max, max_indices)なのでpredは0,1のラベル
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        torch.save(model,f"{anomaly_root}{category}_{8+epoch}.pth")
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model,f"{anomaly_root}{category}_best.pth")

In [ ]:
for c in ["照明"]:
    tl(c)

['その他', '補修不要', '要補修']
Layer (type:depth-idx)                                  Param #
EfficientNet                                            --
├─Sequential: 1-1                                       --
│    └─Conv2dNormActivation: 2-1                        --
│    │    └─Conv2d: 3-1                                 (648)
│    │    └─BatchNorm2d: 3-2                            (48)
│    │    └─SiLU: 3-3                                   --
│    └─Sequential: 2-2                                  --
│    │    └─FusedMBConv: 3-4                            (5,232)
│    │    └─FusedMBConv: 3-5                            (5,232)
│    └─Sequential: 2-3                                  --
│    │    └─FusedMBConv: 3-6                            (25,632)
│    │    └─FusedMBConv: 3-7                            (92,640)
│    │    └─FusedMBConv: 3-8                            (92,640)
│    │    └─FusedMBConv: 3-9                            (92,640)
│    └─Sequential: 2-4                          

## efficientnet_v2

In [ ]:
from tensorflow import keras
def tl(category):
    # 画像ディレクトリ
    IMG_DIR = anomaly_data_path+category
    # バッチサイズ
    BATCH_SIZE = 300
    # VGG16を使用するため以下のサイズに設定
    IMAGE_SIZE = (224, 224)
    IMAGE_SHAPE = IMAGE_SIZE + (3, )

    # 実行毎に同一の結果が得られるようシード値を固定
    RANDOM_STATE = 123

    # エポック数
    EPOCHS = 10

    num_classes=3

    train_ds = tf.keras.utils.image_dataset_from_directory(
    IMG_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    label_mode='categorical',#categorical_crossentropyの時はcategorical
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

    print(train_ds.class_names)

    val_ds=tf.keras.utils.image_dataset_from_directory(
    IMG_DIR,
    validation_split=0.2,
    subset="training",
    label_mode='categorical',
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

    print(train_ds.class_names)

    base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False, weights="imagenet")
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    # 学習させる部分の指定
    layer_names = [l.name for l in base_model.layers]
    idx = layer_names.index('block6a_expand_conv')

    base_model.trainable = True
    for layer in base_model.layers[:idx]:
        layer.trainable = False

    model = tf.keras.Model(base_model.input, predictions)
    #model=keras.models.load_model(os.path.join(f"{anomaly_root}{category}_3.h5"))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=["accuracy"]
    )

    for i in range(EPOCHS):
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            batch_size=BATCH_SIZE, 
            epochs=1, 
            shuffle=True
        )
        print(history)
        print(i)
        model.save(f"{anomaly_root}{category}_{i}.h5")

In [ ]:
for c in ["区画線"]:
    tl(c)

Found 30743 files belonging to 3 classes.
Using 24595 files for training.
['その他', '補修不要', '要補修']
Found 30743 files belonging to 3 classes.
Using 24595 files for training.
['その他', '補修不要', '要補修']
55/82 [===================>..........] - ETA: 51:39 - loss: 0.0760 - accuracy: 0.9719

In [ ]:
for c in ["道路標識","照明","区画線"]:
    tl(c)

Found 4986 files belonging to 3 classes.
Using 3989 files for training.
['その他', '補修不要', '要補修']


Found 4986 files belonging to 3 classes.
Using 3989 files for training.
['その他', '補修不要', '要補修']
9406464/9406464 [==============================] - 0s 0us/step


ValueError: ignored

### base

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# 画像ディレクトリ
category="区画線"
IMG_DIR = anomaly_data_path+category
# バッチサイズ
BATCH_SIZE = 300
# VGG16を使用するため以下のサイズに設定
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3, )

# 実行毎に同一の結果が得られるようシード値を固定
RANDOM_STATE = 123

# エポック数
EPOCHS = 2

num_classes=2

train_ds = tf.keras.utils.image_dataset_from_directory(
  IMG_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode='categorical',#categorical_crossentropyの時はcategorical
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE)

val_ds=tf.keras.utils.image_dataset_from_directory(
  IMG_DIR,
  validation_split=0.2,
  subset="training",
  label_mode='categorical',
  seed=123,
  image_size=IMAGE_SIZE,
  batch_size=BATCH_SIZE)

Found 24872 files belonging to 2 classes.
Using 19898 files for training.
Found 24872 files belonging to 2 classes.
Using 19898 files for training.


In [ ]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False, weights="imagenet")
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# 学習させる部分の指定
layer_names = [l.name for l in base_model.layers]
idx = layer_names.index('block6a_expand_conv')

base_model.trainable = True
for layer in base_model.layers[:idx]:
   layer.trainable = False

model = tf.keras.Model(base_model.input, predictions)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS, 
    shuffle=True
)

Epoch 1/2
67/67 [==============================] - 176s 2s/step - loss: 0.3028 - accuracy: 0.8664 - val_loss: 0.1534 - val_accuracy: 0.9417
Epoch 2/2
67/67 [==============================] - 97s 1s/step - loss: 0.1567 - accuracy: 0.9353 - val_loss: 0.0937 - val_accuracy: 0.9628


In [ ]:
model.save(f"model_{category}.h5")

In [ ]:
from tensorflow import keras
model=keras.models.load_model(f"model_{category}.h5")

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=BATCH_SIZE, 
    epochs=1, 
    shuffle=True
)

67/67 [==============================] - 96s 1s/step - loss: 0.0755 - accuracy: 0.9718 - val_loss: 0.0403 - val_accuracy: 0.9861


In [ ]:
model.save(f"model_{category}_4_.h5")

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=BATCH_SIZE, 
    epochs=1, 
    shuffle=True
)
model.save(f"model_{category}_5_.h5")

67/67 [==============================] - 96s 1s/step - loss: 0.0538 - accuracy: 0.9799 - val_loss: 0.0225 - val_accuracy: 0.9928


## CutPaste

In [ ]:
%cd /content/drive/MyDrive/signate/ワールドＡＩ/anomaly
!git clone https://github.com/LilitYolyan/CutPaste.git
%cd CutPaste
!pip install -r requirements.txt

/content/drive/MyDrive/signate/ワールドＡＩ/anomaly
Cloning into 'CutPaste'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 614 (delta 0), reused 0 (delta 0), pack-reused 606
Receiving objects: 100% (614/614), 1.85 MiB | 9.90 MiB/s, done.
Resolving deltas: 100% (335/335), done.
/content/drive/MyDrive/signate/ワールドＡＩ/anomaly/CutPaste
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.2/526.2 KB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 39.3 MB/s e

In [ ]:
!pip install torch==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.2 requires torch==1.10.1, but you have torch 1.13.1 which is incompatible.


In [ ]:
!python train.py --dataset_path /content/drive/MyDrive/signate/ワールドＡＩ/anomaly/anomaly_data/区画線 --num_class 2

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")
Traceback (most recent call last):
  File "train.py", line 5, in <module>
    import pytorch_lightning as pl
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/__init__.py", line 20, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/__init__.py", line 14, in <module>
    from pytorch_lightning.callbacks.base import Callback
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/base.py", line 26, in <module>
    from pytorch_lightning.utilities.types import STEP_OUTPUT
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lig